### Install packages

In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### Transport many example (with GPU)

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("p208p2002/bart-squad-qg-hl")
model = AutoModelForSeq2SeqLM.from_pretrained("p208p2002/bart-squad-qg-hl").to(device)

Some weights of the model checkpoint at p208p2002/bart-squad-qg-hl were not used when initializing BartForConditionalGeneration: ['lm_head.weight']
- This IS expected if you are initializing BartForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import torch
torch.cuda.is_available()
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
import json
from tqdm import tqdm
result = {}
result['data'] = []
with open('qg_test.json','r') as readfile:
    qg = json.load(readfile)
for d in tqdm(qg['data']):
    temp = {}
    temp['context'] = d['context']
    temp['qas'] = []
    for e in d['input']:
        ans = {}
        input_text = e['sentence']
        question = model.generate(**tokenizer(input_text,return_tensors='pt').to(device))
        ans['text'] = e['answer']
        q = tokenizer.batch_decode(question)
        ans['question'] = q[0][4:len(q[0])-4]
        temp['qas'].append(ans)
    result['data'].append(temp)
with open('qa_dataset-1.json','w') as writefile:
    json.dump(result,writefile)

### Single Example

In [3]:
input_text = 'Architecturally, the school has a[HL]Catholic[HL]character. Atop the Main Building\' s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend" Venite Ad Me Omnes" . Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared'

In [5]:
result = model.generate(**tokenizer(input_text,return_tensors='pt').to(device))

In [6]:
tokenizer.batch_decode(result)

['</s>What type of character is the school?</s>']